In [1]:
import uproot
import numpy as np
import torch
from collections import defaultdict
from util import get_layer, theta_func,create_layer_map
from reco import calculate_num_pixels_z_dependence
import matplotlib.pyplot as plot
import time
from collections import defaultdict
# Get device to be used
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import os
def checkdir(path):
    if not os.path.exists(path): 
        os.makedirs(path)
from IPython.display import clear_output
from tqdm import tqdm

Using device cuda:0


In [2]:
layer_map, super_layer_map = create_layer_map()
x_hit_pos = []
z_pos_list = []
layer_list = []

def process_root_file(file_path):
    print("began processing")
    with uproot.open(file_path) as file:
        tree = file["events/HcalBarrelHits"]
        
        
        z_pos = tree["HcalBarrelHits.position.z"].array(library="np")
        x_pos = tree["HcalBarrelHits.position.x"].array(library="np")
        energy = tree["HcalBarrelHits.EDep"].array(library="np")
        momentum_x = tree["HcalBarrelHits.momentum.x"].array(library="np")
        momentum_y = tree["HcalBarrelHits.momentum.y"].array(library="np")
        momentum_z = tree["HcalBarrelHits.momentum.z"].array(library="np")
        hit_time = tree["HcalBarrelHits.time"].array(library="np")
        mc_hit_idx = file["events/_HcalBarrelHits_MCParticle/_HcalBarrelHits_MCParticle.index"].array(library="np")  # Add PDG code for particle identification
        print("finished loading branches")
        
        processed_data = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))
        
        for event_idx in range(len(z_pos)):
            energy_per_layer_particle = defaultdict(lambda: defaultdict(float))
            first_hit_per_layer_particle = defaultdict(dict)
            
            # First pass: collect first hit data and calculate energy per layer per particle
            for hit_idx in range(len(z_pos[event_idx])):
                z = z_pos[event_idx][hit_idx]
                x = x_pos[event_idx][hit_idx]
                e = energy[event_idx][hit_idx]
                momentum = (momentum_x[event_idx][hit_idx],
                            momentum_y[event_idx][hit_idx],
                            momentum_z[event_idx][hit_idx])
                momentum_mag = np.linalg.norm(momentum)
                theta = theta_func(momentum_x[event_idx][hit_idx], momentum_y[event_idx][hit_idx], momentum_z[event_idx][hit_idx])
                layer = get_layer(x)
                particle_id = mc_hit_idx[event_idx][hit_idx]
                
                energy_per_layer_particle[layer][particle_id] += e
                
                if layer not in first_hit_per_layer_particle or particle_id not in first_hit_per_layer_particle[layer]:
                    first_hit_per_layer_particle[layer][particle_id] = {
                        "z_pos": z,
                        "x_pos": x,
                        "momentum": momentum_mag,
                        "theta": theta,
                        "time": hit_time[event_idx][hit_idx],
                        "mc_hit_idx": particle_id
                    }
            
            
            # Second pass: process first hit with total layer energy per particle
            for layer, particle_data in first_hit_per_layer_particle.items():
                for particle_id, hit_data in particle_data.items():
                    layer_particle_energy = energy_per_layer_particle[layer][particle_id]
                    num_pixels = calculate_num_pixels_z_dependence(layer_particle_energy, hit_data["z_pos"])
#                     print(f"layer:\t\t{layer}\t|\tparticle id:\t{particle_id}\t|\tnum_pixels:\t{num_pixels}")
                    hit_data["num_pixels"] = int(np.floor(num_pixels))
                    hit_data["layer_energy"] = layer_particle_energy  # Store total layer energy for this particle
                    processed_data[event_idx][layer][particle_id] = hit_data
    
    print("finished processing")
    return processed_data

In [3]:
pref = "/hpc/group/vossenlab/rck32/"
# processed_data = process_root_file(pref + "eic/work_eic/root_files/September_3/sector_scint/run_1_pip_0_8_10GeV_theta_90_500events.edm4hep.root")
processed_data = process_root_file(pref + "eic/work_eic/root_files/September_3/sector_scint/run_1_mum_0_8_10GeV_theta_90_500events.edm4hep.root")
# processed_data = process_root_file(pref + "eic/work_eic/root_files/September_3/sector_scint/run_1_n_0_8_10GeV_theta_90_5kevents.edm4hep.root")
# nn_input, nn_output = prepare_nn_input(processed_data, normalizing_flow_model)
# prediction_input, prediction_output = prepare_prediction_input(nn_input, nn_output)

began processing
finished loading branches
finished processing


In [4]:
import normflows as nf
import datetime
x = datetime.datetime.now()
today = x.strftime("%B_%d")
# particle = "pi"
particle = "mu"

# This data will be used for storing plots - today above works if you actually want today
# today = "July_26"

run_num = 7
run_num_str = str(run_num)

#NF Stuff

K = 8 #num flows

latent_size = 1 #dimension of PDF
hidden_units = 256 #nodes in hidden layers
hidden_layers = 26
context_size = 3 #conditional variables for PDF
num_context = 3

K_str = str(K)
batch_size= 2000
hidden_units_str = str(hidden_units)
hidden_layers_str = str(hidden_layers)
batch_size_str = str(batch_size)
flows = []
for i in range(K):
    flows += [nf.flows.AutoregressiveRationalQuadraticSpline(latent_size, hidden_layers, hidden_units, 
                                                             num_context_channels=context_size)]
    flows += [nf.flows.LULinearPermute(latent_size)]

# Set base distribution
q0 = nf.distributions.DiagGaussian(1, trainable=False)
    
# Construct flow model
model = nf.ConditionalNormalizingFlow(q0, flows)

# Move model on GPU if available
model = model.to(device)
# model_date = "August_03"
# today = "August_03"
# model_path = "models/" + model_date + "/"
# checkdir(model_path)

model_path = "/hpc/group/vossenlab/rck32/NF_time_res_models/"

samples_path = "data/samples/" + today + "/"
checkdir(samples_path)

test_data_path = "data/test/" + today + "/"
checkdir(test_data_path)

test_dist_path = "plots/test_distributions/" + today + "/"
checkdir(test_dist_path)
model.load(model_path + "run_" + run_num_str + "_" + str(num_context)+ "context_" +K_str +  "flows_" + hidden_layers_str+"hl_" + hidden_units_str+"hu_" + batch_size_str+"bs.pth")
model = model.to(device)
model_compile = torch.compile(model,mode = "reduce-overhead")
model_compile = model_compile.to(device)

/hpc/group/vossenlab/rck32/ML_venv/lib64/python3.9/site-packages/normflows/core.py:213: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(path))


In [61]:
def prepare_nn_input(processed_data, normalizing_flow, batch_size=1024):
    flattened_data = []
    event_indices = []
    layer_indices = []
    particle_indices = []

    final_event_indices = []
    final_layer_indices = []
    final_particle_indices = []
    
    momentum_list = []
    
    context_list = []
    running_pixel_idx = 0
    for event_idx, event_data in processed_data.items():
        primary_momentum = event_data[0][0]["momentum"]
        for layer, particle_data in event_data.items():
            for particle_id, hit_data in particle_data.items():
                context = torch.tensor([hit_data['z_pos'], hit_data['theta'], hit_data['momentum']], dtype=torch.float32).repeat(hit_data['num_pixels'], 1)
                flattened_data.append(torch.tensor([hit_data['time'], hit_data['num_pixels']]).repeat(hit_data['num_pixels'],1))
                context_list.append(context)
                for pixel_repeat_idx in range(hit_data['num_pixels']):
                    final_event_indices.append(event_idx)
                    final_layer_indices.append(layer)
                    final_particle_indices.append(particle_id)
                    momentum_list.append(primary_momentum.item())
                
#     final_event_indices = torch.cat(event_indices)
#     final_layer_indices = torch.cat(layer_indices)
#     final_particle_indices = torch.cat(particle_indices)
    all_context = torch.cat(context_list).to(device)
    all_time_pixels = torch.cat(flattened_data)
    # Batch the flattened data
    max_its = int(np.ceil(all_context.shape[0] / batch_size))
    sampled_data = []
    for batch_idx in tqdm(range(max_its)):
        begin = batch_idx * batch_size
        data_left = all_context.shape[0] - (batch_idx * batch_size)
        end = min(begin + batch_size,begin + data_left)
        add_times = all_time_pixels[begin:end]
        context_batch = all_context[begin:end].to(device)
        with torch.no_grad():
            samples = abs(normalizing_flow.sample(num_samples=context_batch.shape[0], context=context_batch)[0]).squeeze(1)
        adjusted_times = samples.detach().cpu() + add_times[:,0]
        sampled_data.extend(adjusted_times)
    # Reorganize sampled data
    nn_input = defaultdict(lambda: defaultdict(list))
    nn_output = defaultdict(lambda: defaultdict(list))
    
    for i, (event, layer, particle) in enumerate(zip(final_event_indices, final_layer_indices, final_particle_indices)):
        nn_input[event][layer].append(sampled_data[i])
        nn_output[event][layer].append(torch.Tensor([momentum_list[i]]))
    return nn_input, nn_output

In [62]:
begin = time.time()
nn_input, nn_output = prepare_nn_input(processed_data, model_compile,batch_size = 50000)
end = time.time()
print(f"rate: {(end - begin) / 500} seconds / event")

100%|██████████| 30/30 [00:15<00:00,  1.90it/s]


rate: 0.05313160228729248 seconds / event


In [83]:
def prepare_prediction_input(nn_input, nn_output):
    prediction_input = torch.empty(len(nn_input),28,10)
    prediction_output = torch.empty(len(nn_input))
    
    input_dict = defaultdict(lambda: defaultdict(list))
    output_dict = {}
    for event_idx in tqdm(list(nn_input)):
        event_input = []
        output_dict[event_idx] = nn_output[event_idx][0][0]
        prediction_output[event_idx] = nn_output[event_idx][0][0]
        for layer in nn_input[event_idx].keys():
            layer_times = torch.tensor(sorted(nn_input[event_idx][layer]))
#             print(layer_times[:10])
#             return
            # Pad or truncate to exactly 10 times per layer
            if len(layer_times) < 10:
                padding = torch.full((10 - len(layer_times),), float('inf'))
                layer_times = torch.cat([layer_times, padding])
            
            input_dict[event_idx][layer] = layer_times[:10]
            prediction_input[event_idx][layer] = layer_times[:10]
    return prediction_input, prediction_output


In [84]:
prediction_input, prediction_output= prepare_prediction_input(nn_input,nn_output)

100%|██████████| 500/500 [00:24<00:00, 20.45it/s]


In [ ]:
from reco import Predictor
model = Predictor(input_size=56, num_classes=1, hidden_dim = 512, num_layers = 20)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-5)

In [88]:
#train loop
train_batch_size = 128
total_data_points = prediction_output.shape[0]
max_train_itr = total_data_points / train_batch_size
for batch_idx in tqdm(range(max_train_itr)):
    begin = batch_idx * train_batch_size
    end = min((begin + train_batch_size),(total_data_points - begin))
    batch_inputs = prediction_input[begin:end].flatten()
    batch_outputs = prediction_output[begin:end]
    

tensor(1.0971)